In [1]:
import openai
import requests
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()      

True

In [2]:
openai.api_key = os.getenv("API_KEY")

In [3]:
data = pd.read_csv("data.csv", encoding= 'unicode_escape')

In [4]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [8]:
urls = []
detailed_descs = []
i=0
for desc in data['Description'][5:500]:
    response = openai.Image.create(
      prompt=desc,
      n=3,
      size="512x512"
    )
    current_urls = [x['url'] for x in response['data']]
    urls.append(current_urls)
    
    with open('openai_imgs.txt', 'a') as f:
      for img in current_urls:
        f.write("%s\n" % img)
      f.write("\n")

    text_resp = openai.Completion.create(
      model="text-davinci-001",
      prompt=desc,
      temperature=0.4,
      max_tokens=64,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    current_desc = text_resp['choices'][0]['text']
    detailed_descs.append(current_desc)

    with open('openai_desc.txt', 'a') as f:
      f.write("%s\n" % current_desc)
    i+=1
    print(i)


1
2
3
4
5
6


RateLimitError: Rate limit reached for images per minute. Limit: 20/1min. Current: 21/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.

In [31]:
detailed_descs

['\n\nThis pretty white hanging heart T-light holder is perfect for adding a touch of romance to any room. It has a metal frame with a heart-shaped cut-out and a white fabric cover. There is a loop at the top for hanging and a metal T-light holder at the bottom.',
 '\n\nThis lantern is made of white metal. It is in the shape of a cylinder with a handle on top. There is a small window on one side, and a metal ring on the top.\n\nWhen placed on a flat surface, the lantern stands about 6 inches tall.\n\nThis lantern casts a']

### Cheap recommendation 

In [40]:
data['CustomerID'].value_counts().values[:100]

array([7983, 5903, 5128, 4642, 2782, 2491, 2085, 1857, 1677, 1640, 1508,
       1420, 1284, 1212, 1165, 1160, 1152, 1128, 1094, 1076, 1061, 1011,
        981,  977,  938,  925,  898,  872,  803,  778,  757,  756,  742,
        741,  721,  720,  718,  710,  705,  702,  696,  696,  690,  674,
        671,  668,  662,  653,  646,  633,  632,  631,  612,  612,  605,
        604,  590,  588,  585,  582,  576,  566,  565,  563,  561,  560,
        558,  551,  548,  547,  547,  546,  533,  531,  525,  523,  519,
        516,  513,  509,  506,  505,  503,  502,  501,  501,  501,  500,
        495,  489,  487,  487,  481,  479,  474,  474,  474,  472,  472,
        470], dtype=int64)

In [41]:
data['InvoiceNo'].value_counts().values[:100]

array([1114,  749,  731,  721,  705,  687,  676,  675,  662,  652,  650,
        645,  635,  629,  622,  620,  606,  601,  597,  593,  593,  592,
        591,  585,  578,  572,  568,  561,  552,  544,  542,  541,  536,
        534,  533,  529,  527,  527,  526,  526,  520,  518,  518,  518,
        515,  512,  503,  503,  502,  494,  490,  487,  486,  482,  474,
        464,  462,  460,  458,  455,  450,  444,  442,  442,  438,  436,
        435,  434,  429,  421,  421,  418,  416,  416,  415,  411,  410,
        406,  404,  403,  402,  399,  388,  384,  379,  378,  377,  371,
        364,  360,  359,  355,  353,  353,  352,  352,  350,  347,  346,
        344], dtype=int64)

In [75]:
word = "wallet"
similar_items = []
for i in range(len(data)):
    if word in str(data.iloc[i]['Description']).lower():
        vals = data[data['InvoiceNo'] == data.iloc[i]['InvoiceNo']]['Description'].values
        similar_items.append(vals)
        break
        

In [76]:
similar_items

[array(['TRAVEL CARD WALLET FLOWER MEADOW', 'TRAVEL CARD WALLET PANTRY',
        'TRAVEL CARD WALLET SKULLS', 'TRAVEL CARD WALLET VINTAGE ROSE ',
        'TRAVEL CARD WALLET TRANSPORT', 'TRAVEL CARD WALLET RETROSPOT',
        'TRAVEL CARD WALLET KEEP CALM', 'SUNFLOWER DECORATIVE PARASOL',
        'POLYESTER FILLER PAD 45x45cm', 'SET OF 6 VINTAGE NOTELETS KIT',
        'PACK OF 6 BIRDY GIFT TAGS', 'BEADED CRYSTAL HEART BLUE SMALL',
        'BEADED CRYSTAL HEART BLUE  LARGE', 'ASSORTED COLOURS SILK FAN',
        'ENGLISH ROSE HOT WATER BOTTLE'], dtype=object)]

In [35]:
data['CustomerID'].describe()

count    406829.000000
mean      15287.690570
std        1713.600303
min       12346.000000
25%       13953.000000
50%       15152.000000
75%       16791.000000
max       18287.000000
Name: CustomerID, dtype: float64